## 1. Load the tips dataset.

In [34]:
from pydataset import data
tips = data("tips")

In [30]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


### a. Create a column named price_per_person. This should be the total bill divided by the party size.

In [35]:
tips['price_per_person'] = tips.total_bill / tips['size']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


### b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

* size ? time? 

In [37]:
tips = pd.get_dummies(tips, ['sex', 'smoker', 'day', 'time'], drop_first=True)
tips.head()

,total_bill,tip,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
1,16.99,1.01,2,8.495000,0,0,0,1,0,0
2,10.34,1.66,3,3.446667,1,0,0,1,0,0
3,21.01,3.50,3,7.003333,1,0,0,1,0,0
4,23.68,3.31,2,11.840000,1,0,0,1,0,0
5,24.59,3.61,4,6.147500,0,0,0,1,0,0


### c. Use select k best to select the top 2 features for predicting tip amount. What are they?

In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.model_selection import train_test_split

In [38]:
train_validate, test = train_test_split(tips, random_state=123, train_size=.8)
train, validate = train_test_split(train_validate, random_state=123, train_size=.7)
train.shape, validate.shape, test.shape

((136, 10), (59, 10), (49, 10))

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 19 to 167
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        136 non-null    float64
 1   tip               136 non-null    float64
 2   size              136 non-null    int64  
 3   price_per_person  136 non-null    float64
 4   sex_Male          136 non-null    uint8  
 5   smoker_Yes        136 non-null    uint8  
 6   day_Sat           136 non-null    uint8  
 7   day_Sun           136 non-null    uint8  
 8   day_Thur          136 non-null    uint8  
 9   time_Lunch        136 non-null    uint8  
dtypes: float64(3), int64(1), uint8(6)
memory usage: 6.1 KB


In [40]:
x_train = train.drop(columns=['tip'])
y_train = train.tip
x_validate = validate.drop(columns=['tip'])
y_validate = validate.tip
x_test = test.drop(columns=['tip'])
y_test = test.tip

In [41]:
x_train.head()

,total_bill,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
19,16.97,3,5.656667,0,0,0,1,0,0
173,7.25,2,3.625000,1,1,0,1,0,0
119,12.43,2,6.215000,0,0,0,0,1,1
29,21.70,2,10.850000,1,0,1,0,0,0
238,32.83,2,16.415000,1,1,1,0,0,0


In [42]:
y_train[:5]

19     3.50
173    5.15
119    1.80
29     4.30
238    1.17
Name: tip, dtype: float64

In [43]:
kbest = SelectKBest(f_regression, k=2)
kbest.fit(x_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7ff060d8c3a0>)

In [44]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=x_train.columns)
kbest_results

,p,f
total_bill,7.186470e-20,115.984909
size,1.341642e-12,61.259089
price_per_person,1.310327e-03,10.777792
sex_Male,2.844794e-01,1.154792
smoker_Yes,5.579978e-01,0.344909
day_Sat,9.550468e-01,0.003190
day_Sun,1.236625e-01,2.400404
day_Thur,3.013774e-01,1.076380
time_Lunch,1.821449e-01,1.798647


In [46]:
x_train.columns[kbest.get_support()]

Index(['total_bill', 'size'], dtype='object')

In [47]:
x_train_transformed = pd.DataFrame(
    kbest.transform(x_train),
    index=x_train.index,
    columns=x_train.columns[kbest.get_support()]
)
x_train_transformed.head()

,total_bill,size
19,16.97,3.0
173,7.25,2.0
119,12.43,2.0
29,21.70,2.0
238,32.83,2.0


### d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [48]:
from sklearn.linear_model import LinearRegression

In [49]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(x_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [50]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=x_train.columns)

,rfe_ranking
total_bill,5
size,1
price_per_person,3
sex_Male,2
smoker_Yes,7
day_Sat,4
day_Sun,6
day_Thur,1
time_Lunch,8


In [51]:
x_train.columns[rfe.get_support()]

Index(['size', 'day_Thur'], dtype='object')

In [52]:
x_train_transformed = pd.DataFrame(
    rfe.transform(x_train),
    index=x_train.index,
    columns=x_train.columns[rfe.support_]
)
x_train_transformed.head()

,size,day_Thur
19,3.0,0.0
173,2.0,0.0
119,2.0,1.0
29,2.0,0.0
238,2.0,0.0


### e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [54]:
rfe1 = RFE(model, n_features_to_select=4)
rfe1.fit(x_train, y_train)
x_train.columns[rfe1.get_support()]

Index(['size', 'price_per_person', 'sex_Male', 'day_Thur'], dtype='object')

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [55]:
from sklearn.feature_selection import SelectKBest, RFE, f_regression

def select_kbest(x, y, i):
    kbest = SelectKBest(f_regression, k=i)
    kbest.fit(x, y)
    return x.columns[kbest.get_support()]

In [56]:
select_kbest(x_train, y_train, 2)

Index(['total_bill', 'size'], dtype='object')

### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [57]:
from sklearn.linear_model import LinearRegression

def rfe(x, y, i):
    
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=i)
    rfe.fit(x, y)
    
    return x.columns[rfe.get_support()]


In [58]:
rfe(x_train, y_train, 2)

Index(['size', 'day_Thur'], dtype='object')

### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [59]:
swiss = data("swiss")
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [61]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [62]:
train_validate1, test1 = train_test_split(swiss, random_state=123, train_size=.8)
train1, validate1 = train_test_split(train_validate1, random_state=123, train_size=.7)
train1.shape, validate1.shape, test1.shape

((25, 6), (12, 6), (10, 6))

In [64]:
x_train1 = train1.drop(columns=['Fertility'])
y_train1 = train1.Fertility
x_validate1 = validate1.drop(columns=['Fertility'])
y_validate1 = validate1.Fertility
x_test1 = test1.drop(columns=['Fertility'])
y_test1 = test1.Fertility

In [65]:
select_kbest(x_train1, y_train1, 3)

Index(['Examination', 'Catholic', 'Infant.Mortality'], dtype='object')

In [66]:
rfe(x_train1, y_train1, 3)

Index(['Agriculture', 'Examination', 'Infant.Mortality'], dtype='object')